In [1]:
import requests
from bs4 import BeautifulSoup

# URL of the form
url = "https://www.dmr.nd.gov/oilgas/findwellsvw.asp"

# Form data - ONLY Township 129 selected
payload = {
    "ddmTownship": "129",
    "ddmSection": "0",
    "ddmRange": "0",
    "B1": "Submit"
}

# Send POST request to the form
response = requests.post(url, data=payload)

print("Status code:", response.status_code)
print("Page length:", len(response.text))

# Look at a snippet from the *middle* of the page (helpful for debugging)
mid = len(response.text) // 2
print("\nMIDDLE 1000 CHARACTERS:\n")
print(response.text[mid:mid+1000])


Status code: 200
Page length: 434507

MIDDLE 1000 CHARACTERS:

an title="Active">A</span></td><td style="border:1px solid saddlebrown;">7/3/2024</td><td style="border:1px solid saddlebrown;" align="right">14865</td><td style="border:1px solid saddlebrown;">SESE 26-129-102</td><td style="border:1px solid saddlebrown;">LUFF EXPLORATION COMPANY</td><td style="border:1px solid saddlebrown;">SANDY WHITE P-26H</td><td style="border:1px solid saddlebrown;">COREY BUTTE</td></tr>
<tr><td style="border:1px solid saddlebrown;" align="right">14546</td><td style="border:1px solid saddlebrown;" align="right"></td><td style="border:1px solid saddlebrown;">3301100832</td><td style="border:1px solid saddlebrown;"><span title="Oil & Gas">OG</span></td><td style="border:1px solid saddlebrown;"><span title="Permit Now Cancelled">PNC</span></td><td style="border:1px solid saddlebrown;">12/18/1997</td><td style="border:1px solid saddlebrown;">&#xa0;</td><td style="border:1px solid saddlebrown;">NENE 27-129-

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "html.parser")

# Find ALL tables
tables = soup.find_all("table")
print("Number of tables found:", len(tables))

# Print number of rows in each table so we can find the correct one
for i, t in enumerate(tables):
    row_count = len(t.find_all("tr"))
    print(f"Table {i}: {row_count} rows")

# Now use the table with MANY rows (likely the data table)
for t in tables:
    if len(t.find_all("tr")) > 5:   # data table will have lots of rows
        data_table = t
        break

rows = data_table.find_all("tr")[1:]  # skip header

print("\n✅ Data rows found:", len(rows))

# Show first 3 rows
for row in rows[:3]:
    cells = [td.get_text(strip=True) for td in row.find_all("td")]
    print(cells)


Number of tables found: 3
Table 0: 2 rows
Table 1: 1 rows
Table 2: 378 rows

✅ Data rows found: 377
['99025', '', '3302199025', 'ST', 'DRY', '9/30/1963', '1600', 'NENE 10-129-63', 'JACK ROUSE OIL PROPERTIES', 'HAROLD BILLIE 1', 'WILDCAT']
['515', '', '3302100001', 'OG', 'DRY', '11/17/1954', '1882', 'SENW 11-129-63', 'H. HANSON OIL SYNDICATE', 'HAROLD BILLEY 1', 'WILDCAT']
['572', '', '3302100002', 'OG', 'DRY', '6/17/1954', '1860', 'NENW 14-129-63', 'H. HANSON OIL SYNDICATE', 'JOHN BELL 1', 'WILDCAT']


In [9]:
import csv

filename = "/Users/nataliejonas/Desktop/township_129_wells.csv"

print("Rows to be written:", len(clean_data))

with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(clean_data)

print("✅ Saved to Desktop:", filename)


Rows to be written: 377
✅ Saved to Desktop: /Users/nataliejonas/Desktop/township_129_wells.csv


In [13]:
import requests
from bs4 import BeautifulSoup
import csv
from time import sleep
from tqdm import tqdm  # standard progress bar, works in VS Code

# Township values
townships = list(range(129, 165))  # 129 → 164 inclusive

# CSV setup
filename = "/Users/nataliejonas/Desktop/all_townships_wells.csv"

headers = [
    "File No",
    "CTB No",
    "API No",
    "Well Type",
    "Well Status",
    "Status Date",
    "DTD",
    "Location",
    "Operator",
    "Well Name",
    "Field"
]

all_data = []

# Loop through each township
for twp in tqdm(townships, desc="Scraping townships"):
    payload = {
        "ddmTownship": str(twp),
        "ddmSection": "0",
        "ddmRange": "0",
        "B1": "Submit"
    }
    
    response = requests.post("https://www.dmr.nd.gov/oilgas/findwellsvw.asp", data=payload)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find the data table (the one with >5 rows)
    tables = soup.find_all("table")
    data_table = None
    for t in tables:
        if len(t.find_all("tr")) > 5:
            data_table = t
            break
    
    if data_table is None:
        print(f"No data table found for township {twp}")
        continue
    
    # Extract rows
    rows = data_table.find_all("tr")[1:]
    for row in rows:
        cells = [td.get_text(strip=True) for td in row.find_all("td")]
        if len(cells) == len(headers):
            all_data.append(cells)
    
    sleep(0.5)  # polite pause to avoid overloading the server

# Write all data to CSV
with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(all_data)

print("\n✅ Done! Total rows:", len(all_data))
print("✅ File saved:", filename)


Scraping townships:   0%|          | 0/36 [00:00<?, ?it/s]Exception ignored in: <function tqdm.__del__ at 0x109c0fec0>
Traceback (most recent call last):
  File "/Users/nataliejonas/.pyenv/versions/3.13.9/lib/python3.13/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/Users/nataliejonas/.pyenv/versions/3.13.9/lib/python3.13/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Scraping townships: 100%|██████████| 36/36 [01:40<00:00,  2.80s/it]


✅ Done! Total rows: 43132
✅ File saved: /Users/nataliejonas/Desktop/all_townships_wells.csv
